In [35]:
!pip install mlxtend

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn import linear_model, decomposition, datasets
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import math
from mlxtend.plotting import plot_confusion_matrix

In [44]:
#retrieving the data
parkingdata = pd.read_csv("parking_duration_of_parking_event_vs_street_ID.csv")
parkingdata = parkingdata.sample(n=10000)
parkingdata.size

130000

In [45]:
#Converting all the feature attributes to uppercase for uniformity

parkingdata['Area Name'] = parkingdata['Area Name'].str.upper()
parkingdata['Street Name'] = parkingdata['Street Name'].str.upper()
parkingdata['Between Street 1'] = parkingdata['Between Street 1'].str.upper()
parkingdata['Between Street 2'] = parkingdata['Between Street 2'].str.upper()
parkingdata['Street Marker'] = parkingdata['Street Marker'].str.upper()
parkingdata['Sign'] = parkingdata['Sign'].str.upper()

parkingdata.head(5)

,Area Name,Street Name,Between Street 1,Between Street 2,Side Of Street,Street Marker,Arrival Time,Departure Time,Duration of Parking Event (in seconds),Sign,In Violation?,Street ID,Device ID
9104418,RIALTO,LT COLLINS STREET,KING STREET,WILLIAM STREET,4,2275S,28/06/2012 05:46:29 PM,28/06/2012 05:49:02 PM,153,1/2P MTR M-SAT 7:30-19:30,0,911,5491
2709291,QUEENSBERRY,VICTORIA STREET,KING STREET,HAWKE STREET,3,7628N,14/12/2011 04:51:00 PM,14/12/2011 04:52:04 PM,64,2P TKT A M-F 7:30-18:30,0,1381,3936
1188767,QUEENSBERRY,WILLIAM STREET,VICTORIA STREET,WALSH STREET,2,5578E,03/11/2011 12:27:50 PM,03/11/2011 12:44:40 PM,1010,2P TKT A M-F 7:30-18:30,0,1428,4194
6684328,PRINCES THEATRE,EXHIBITION STREET,LT BOURKE STREET,BOURKE STREET,1,C466,15/04/2012 04:42:27 PM,15/04/2012 06:30:00 PM,6453,1P SUN 7:30-18:30,1,647,371
638720,QUEENSBERRY,CHETWYND STREET,QUEENSBERRY STREET,VICTORIA STREET,1,C6146,19/10/2011 12:29:38 PM,19/10/2011 12:34:35 PM,297,4P MTR M-F 7:30-18:30,0,192,3387


In [46]:
#Removing all redundant extra whitespaces
for x in parkingdata.columns:
    if parkingdata[x].dtype == object:
        parkingdata[x] = parkingdata[x].str.strip()

In [47]:
#converting object to datetime
parkingdata['Arrival Time'] = pd.to_datetime(parkingdata['Arrival Time'])
parkingdata['Departure Time'] = pd.to_datetime(parkingdata['Departure Time'])

In [48]:
#check which columns have missing values
parkingdata.columns[parkingdata.isnull().any()]

Index([], dtype='object')

In [49]:
#Target field identification
target = parkingdata['Street Name']

unique_vals = target.unique()
target.replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

target.head(10)

9104418     0
2709291     1
1188767     2
6684328     3
638720      4
7035910     5
6662374     6
1587958     7
11756704    8
4602733     6
Name: Street Name, dtype: int64

In [50]:
#Preliminary Target Features identification (Pre-Hill Climbing)

#Convert possible targets to dtype = int
intDf = parkingdata.copy()
#Drop useless fields
intDf.drop('Arrival Time', axis = 1, inplace = True)
intDf.drop('Departure Time', axis = 1, inplace = True)
intDf.drop('Street Marker', axis = 1, inplace = True)
#Drop target field
intDf.drop('Duration of Parking Event (in seconds)', axis = 1, inplace = True)
#Drop already int fields (Will add back after)
intDf.drop('In Violation?', axis = 1, inplace = True)
intDf.drop('Side Of Street', axis = 1, inplace = True)
intDf.drop('Street ID', axis = 1, inplace = True)
intDf.drop('Device ID', axis = 1, inplace = True)

#Transform the remainder non-int fields' values to unique int identifiers
for column in intDf:
    unique_vals = intDf[column].unique()
    intDf[column].replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

#Create dataframe for all the relevant features
features = intDf.copy()

#Add back int fields (If not the target field)
features['In Violation?'] = parkingdata['In Violation?']
features['Side Of Street'] = parkingdata['Side Of Street']
features['Street ID'] = parkingdata['Street ID']
features['Device ID'] = parkingdata['Device ID']
#Add back datetime features with only the meaningful subset of data
features['Arrival Hour'] = parkingdata['Arrival Time'].dt.hour
features['Departure Hour'] = parkingdata['Departure Time'].dt.hour

#Table view of features dataframe
features.head(10)

,Area Name,Street Name,Between Street 1,Between Street 2,Sign,In Violation?,Side Of Street,Street ID,Device ID,Arrival Hour,Departure Hour
9104418,0,0,0,0,0,0,4,911,5491,17,17
2709291,1,1,0,1,1,0,3,1381,3936,16,16
1188767,1,2,1,2,1,0,2,1428,4194,12,12
6684328,2,3,2,3,2,1,1,647,371,16,18
638720,1,4,3,4,3,0,1,192,3387,12,12
7035910,3,5,4,5,4,0,3,123,4952,9,9
6662374,4,6,5,6,5,0,3,894,1824,14,14
1587958,5,7,4,5,4,1,3,926,1681,13,16
11756704,6,8,6,7,4,0,1,1171,5861,10,10
4602733,2,6,7,8,4,0,1,894,1904,11,11


In [51]:
#Parameter Tuning

#Define the parameters to tune and the values to tune to
params_randomforest = [
                { 
                   'n_estimators' : (10,20,30,50,70,90,100),
                    'criterion' : ('gini', 'entropy'),
                    'max_depth' : (3, 5, 7, 9, 10, 15),
                    'max_features' : ('auto', 'sqrt'),
                    'min_samples_split' : (2, 4, 6, 8, 10, 14, 16)
                    
                }
            ]

In [52]:
cv_method = RepeatedStratifiedKFold(n_splits = 5, 
                                    n_repeats = 1, 
                                    random_state = 1)

gs_randomforest = GridSearchCV(RandomForestClassifier(), 
                      param_grid = params_randomforest, 
                      cv = cv_method,
                      verbose = True,
                      scoring = 'accuracy',
                      n_jobs = -1,
                      return_train_score = True)

In [53]:
#Fit the model with the dataset
bestModel = gs_randomforest.fit(features, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 13

In [54]:
gs_randomforest.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 90}

In [55]:
#Find the best values for the  parameters of the model (Formatted output)
best_e = bestModel.best_estimator_.get_params()['n_estimators']
best_C = bestModel.best_estimator_.get_params()['criterion']
best_maxd =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf =  bestModel.best_estimator_.get_params()['max_features']
best_mins =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimator:', best_e)
print('Best criterion:', best_C)
print('Best max depth:', best_maxd)
print('Best max features:', best_maxf)
print('Best min_samples_split:', best_mins)

Best estimator: 90
Best criterion: gini
Best max depth: 15
Best max features: auto
Best min_samples_split: 2


In [56]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5446
1,gini,3,auto,2,20,0.5747
2,gini,3,auto,2,30,0.5689
3,gini,3,auto,2,50,0.6006
4,gini,3,auto,2,70,0.5884
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9995
1172,entropy,15,sqrt,16,50,0.9997
1173,entropy,15,sqrt,16,70,0.9997
1174,entropy,15,sqrt,16,90,0.9997


In [57]:
#Re-Define model with the optimal parameter values AFTER HILL CLIMBING
randomforest = metric = RandomForestClassifier(n_estimators = best_e, 
                               criterion = best_C, 
                               max_depth = best_maxd, 
                               max_features = best_maxf,
                               min_samples_split = best_mins                
                               )

In [58]:
# Hill climbing w/ Randomforest
new_Ind = []
cur_MaxScore = 0.0
col_num = len(features.columns)
col_Ind_Random = shuffle(range(0, col_num), random_state = 1)
features_array = features.values

for cur_f in range(col_num):
    new_Ind.append(col_Ind_Random[cur_f])
    newData = features_array[:, new_Ind]
    x_train, x_test, y_train, y_test = train_test_split(newData, target, test_size=0.2, random_state=1)
    fit = randomforest.fit(x_train, y_train)
    cur_Score = randomforest.score(x_test, y_test)
    
    if cur_Score < cur_MaxScore:
        new_Ind.remove(col_Ind_Random[cur_f])
    else:
        cur_MaxScore = cur_Score
        print ("Score with " + str(len(new_Ind)) + " selected features: " + str(cur_Score))
print("\nIndexs of the desired features")
print(new_Ind)

Score with 1 selected features: 0.384
Score with 2 selected features: 0.458
Score with 3 selected features: 0.7605
Score with 4 selected features: 0.999
Score with 5 selected features: 0.9995
Score with 6 selected features: 0.9995

Indexs of the desired features
[2, 3, 4, 1, 7, 8]


In [59]:
features_hc = pd.DataFrame()
for index in new_Ind:
    colName = features.columns[index]
    features_hc[colName] = features[colName]
features_hc.head(10)

,Between Street 1,Between Street 2,Sign,Street Name,Street ID,Device ID
9104418,0,0,0,0,911,5491
2709291,0,1,1,1,1381,3936
1188767,1,2,1,2,1428,4194
6684328,2,3,2,3,647,371
638720,3,4,3,4,192,3387
7035910,4,5,4,5,123,4952
6662374,5,6,5,6,894,1824
1587958,4,5,4,7,926,1681
11756704,6,7,4,8,1171,5861
4602733,7,8,4,6,894,1904


In [60]:
bestModel = gs_randomforest.fit(features_hc, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 12

In [61]:
gs_randomforest.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_split': 8,
 'n_estimators': 70}

In [62]:
best_e_hc = bestModel.best_estimator_.get_params()['n_estimators']
best_C_hc = bestModel.best_estimator_.get_params()['criterion']
best_maxd_hc =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf_hc =  bestModel.best_estimator_.get_params()['max_features']
best_mins_hc =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimators:', best_e_hc)
print('Best criterion:', best_C_hc)
print('Best max depth:', best_maxd_hc)
print('Best max features:', best_maxf_hc)
print('Best min_samples_split:', best_mins_hc)

Best estimators: 70
Best criterion: gini
Best max depth: 10
Best max features: auto
Best min_samples_split: 8


In [63]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5275
1,gini,3,auto,2,20,0.5508
2,gini,3,auto,2,30,0.5604
3,gini,3,auto,2,50,0.5854
4,gini,3,auto,2,70,0.5709
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9996
1172,entropy,15,sqrt,16,50,0.9994
1173,entropy,15,sqrt,16,70,0.9991
1174,entropy,15,sqrt,16,90,0.9996


In [64]:
randomforest = metric = RandomForestClassifier(n_estimators = best_e_hc, 
                               criterion = best_C_hc, 
                               max_depth = best_maxd_hc, 
                               max_features = best_maxf_hc,
                               min_samples_split = best_mins_hc                
                               )

In [68]:
#Defining training and testing groups
x_train, x_test, y_train, y_test = train_test_split(features_hc, target, test_size = 0.5, random_state = 4)

#Training the model previously defined
randomforest.fit(x_train, y_train)

#Obtaining and printing out results from the model (Confusion Matrix)
predicted = randomforest.predict(x_test)
cm = metrics.confusion_matrix(y_test,predicted)
print (cm)
print (classification_report(y_test,predicted))

[[145   0   0 ...   0   0   0]
 [  0  78   0 ...   0   0   0]
 [  0   0  85 ...   0   0   0]
 ...
 [  0   0   0 ...   6   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       145
           1       1.00      1.00      1.00        78
           2       1.00      1.00      1.00        85
           3       1.00      1.00      1.00       257
           4       1.00      1.00      1.00        34
           5       1.00      1.00      1.00       212
           6       1.00      1.00      1.00       454
           7       1.00      1.00      1.00       165
           8       1.00      1.00      1.00       371
           9       1.00      1.00      1.00        26
          10       1.00      1.00      1.00        22
          11       1.00      1.00      1.00       291
          12       1.00      1.00      1.00       117
          13       1.00      1.00      1.00       1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
#Repeat with KFold - creating groups
kf = KFold(n_splits = 5, random_state = 4, shuffle = True)


#Repeat with KFold - Training model (previously defined) and obtaining its output
kFoldTotal = 0
for k, (train_index, test_index) in enumerate(kf.split(features_hc)):
    x_train, x_test = features_hc.iloc[train_index], features_hc.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    randomforest.fit(x_train, y_train)
    kFoldTotal += randomforest.score(x_test, y_test)
    print("[fold {0}] score: {1:.5f}".format(k, randomforest.score(x_test, y_test)))

#Printing out the results
roundedTotal = round(kFoldTotal/5, 5)
print("\nKNN mean score [5 folds] = " + str(roundedTotal))

[fold 0] score: 1.00000
[fold 1] score: 0.99800
[fold 2] score: 0.99900
[fold 3] score: 0.99850
[fold 4] score: 0.99750

KNN mean score [5 folds] = 0.9986
